# Can only be run on your desktop!!!

In [21]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
import keras
import numpy as np
from utils import *
import os
from keras_self_attention import *
import matplotlib.pyplot as plt
import pypianoroll
from utils import *
from glob import glob
from generator import *
from tqdm import tqdm
import subprocess
import sys

In [23]:
%pwd

'D:\\data\\thesis_model2'

# Load model

In [24]:
MODEL_ID = 'model_hooktheory_attention/'

json_model = open(os.path.join(MODEL_ID,"model.json"),"r").read()

model = keras.models.model_from_json(json_model, custom_objects=SeqWeightedAttention.get_custom_objects())

model.load_weights(os.path.join(MODEL_ID, "model.h5"))

model.summary(line_length=100)

C:\Anaconda3\envs\magenta36_1\lib\site-packages\ipykernel_launcher.py:3: ResourceWarning: unclosed file <_io.TextIOWrapper name='model_hooktheory_attention/model.json' mode='r' encoding='cp1252'>
  This is separate from the ipykernel package so we can avoid doing imports until


____________________________________________________________________________________________________
Layer (type)                                 Output Shape                            Param #        
Input (InputLayer)                           (None, 97, 278)                         0              
____________________________________________________________________________________________________
LSTM (LSTM)                                  (None, 97, 64)                          87808          
____________________________________________________________________________________________________
LSTM_2 (LSTM)                                (None, 97, 64)                          33024          
____________________________________________________________________________________________________
Attention (SeqWeightedAttention)             (None, 64)                              65             
___________________________________________________________________________________________

# Sampling

In [4]:
import shutil

from config import datasets

dshape = datasets['HOOKTHEORY_DATASET_MAGENTA_40k']['shape']

sys.path.append("d:/data/magenta-1.0.2/magenta/models/score2perf/")
sys.path.append("preprocessing/")

import encoders
import my_encoder

import transpose

INPUT_SEQ_LEN = dshape[1]-1
print(INPUT_SEQ_LEN)

C:\Anaconda3\envs\magenta36_1\lib\site-packages\networkx\external\decorator\decorator3\_decorator3.py:203: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  first = inspect.getargspec(caller)[0][0] # first arg
C:\Anaconda3\envs\magenta36_1\lib\site-packages\networkx\external\decorator\decorator3\_decorator3.py:203: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  first = inspect.getargspec(caller)[0][0] # first arg
C:\Anaconda3\envs\magenta36_1\lib\site-packages\networkx\external\decorator\decorator3\_decorator3.py:203: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  first = inspect.getargspec(caller)[0][0] # first arg
C:\Anaconda3\envs\magenta36_1\lib\site-packages\networkx\external\decorator\decorator3\_decorator3.py:203: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or 

C:\Anaconda3\envs\magenta36_1\lib\site-packages\networkx\external\decorator\decorator3\_decorator3.py:203: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  first = inspect.getargspec(caller)[0][0] # first arg
C:\Anaconda3\envs\magenta36_1\lib\site-packages\networkx\external\decorator\decorator3\_decorator3.py:203: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  first = inspect.getargspec(caller)[0][0] # first arg
C:\Anaconda3\envs\magenta36_1\lib\site-packages\networkx\external\decorator\decorator3\_decorator3.py:203: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  first = inspect.getargspec(caller)[0][0] # first arg
C:\Anaconda3\envs\magenta36_1\lib\site-packages\networkx\external\decorator\decorator3\_decorator3.py:203: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or 

97


In [30]:
file = "d:\\data\\sessiontune10010.mid"
unique_name = ''.join(file.split(os.path.sep)[-1].split(".mid")[:-1])

if not os.path.exists('midi_seeds_transposed'):
    os.mkdir('midi_seeds_transposed')

transpose.main([file], os.path.abspath("midi_seeds_transposed/"))

transposed_seed = os.path.abspath(glob(os.path.join("midi_seeds_transposed",unique_name)+"*")[0])
print(transposed_seed)
seq = my_encoder.encoder.encode(transposed_seed)

seed_sequence = keras.utils.to_categorical(seq, dshape[2])
if len(seed_sequence) > INPUT_SEQ_LEN:
    seed_sequence = seed_sequence[:INPUT_SEQ_LEN]
else:
    zero_padded_seq = np.zeros((INPUT_SEQ_LEN, dshape[2]))
    zero_padded_seq[INPUT_SEQ_LEN-len(seq):] = seed_sequence
    seed_sequence = zero_padded_seq

seed_sequence.shape

generated = np.zeros((4*dshape[1], seed_sequence.shape[1]))
print(generated.shape)

generated[:INPUT_SEQ_LEN] = seed_sequence

seed_filename = transposed_seed.split(os.path.sep)[-1].split(".mid")[0]

if not os.path.exists(os.path.join(MODEL_ID, "samples")):
    os.mkdir(os.path.join(MODEL_ID, "samples"))

samples_dir = os.path.abspath(os.path.join(MODEL_ID, "samples", seed_filename))
if not os.path.exists(samples_dir):
    os.mkdir(samples_dir)

seedpath = os.path.join(samples_dir, "1seed.mid")

my_encoder.encoder.decode(remove_zeros_from_onehot(seed_sequence), path=seedpath)

transposing...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 13.14it/s]


D:\data\thesis_model2\midi_seeds_transposed\sessiontune10010_transposed.mid


In [46]:
temperature = 1.5

nr_samples = 1

for i in tqdm(list(range(nr_samples))):
    
    for timestep in range(INPUT_SEQ_LEN, len(generated)):
        start_index = timestep - (INPUT_SEQ_LEN)
        sequence_for_prediction = generated[start_index:timestep]
        next_step = sample(model, sequence_for_prediction, temperature)
        print('step %s: %s' %(timestep, my_encoder.printevents(np.argmax(next_step,axis=1))))
        generated[timestep] = next_step

    new_path = os.path.join(samples_dir, "temp_%s_%s.mid" %(temperature, i))
    path = my_encoder.encoder.decode(remove_zeros_from_onehot(generated[INPUT_SEQ_LEN:]), path=new_path)

#     if nr_samples - i < 4:
    roll = pypianoroll.Multitrack(new_path).tracks[0]

    plt.figure(figsize=(14,8))
    ax = plt.gca()
    pypianoroll.plot_pianoroll(ax, roll.pianoroll)
    plt.show()

  0%|                                                                                                                                                             | 0/1 [00:00<?, ?it/s]

step 97: ["PerformanceEvent('Note OFF', 73)"]
step 98: ["PerformanceEvent('Note OFF', 73)"]
step 99: ["PerformanceEvent('Note OFF', 75)"]
step 100: ["PerformanceEvent('Note OFF', 73)"]
step 101: ["PerformanceEvent('Note OFF', 67)"]
step 102: ["PerformanceEvent('Note OFF', 73)"]
step 103: ["PerformanceEvent('Note OFF', 37)"]
step 104: ["PerformanceEvent('Note OFF', 76)"]
step 105: ["PerformanceEvent('Note OFF', 33)"]
step 106: ["PerformanceEvent('Note OFF', 74)"]
step 107: ["PerformanceEvent('Note OFF', 77)"]
step 108: ["PerformanceEvent('Note OFF', 74)"]
step 109: ["PerformanceEvent('Note OFF', 74)"]
step 110: ["PerformanceEvent('Note OFF', 75)"]
step 111: ["PerformanceEvent('Note OFF', 76)"]
step 112: ["PerformanceEvent('Note OFF', 76)"]
step 113: ["PerformanceEvent('Note OFF', 79)"]
step 114: ["PerformanceEvent('Note OFF', 67)"]
step 115: ["PerformanceEvent('Note OFF', 76)"]
step 116: ["PerformanceEvent('Note OFF', 76)"]
step 117: ["PerformanceEvent('Note OFF', 74)"]
step 118: ["Perf

ValueError: Cannot get beat timings to quantize pianoroll.

In [47]:
my_encoder.printevents(np.argmax(generated[INPUT_SEQ_LEN:],axis=1))

["PerformanceEvent('Note OFF', 73)",
 "PerformanceEvent('Note OFF', 73)",
 "PerformanceEvent('Note OFF', 75)",
 "PerformanceEvent('Note OFF', 73)",
 "PerformanceEvent('Note OFF', 67)",
 "PerformanceEvent('Note OFF', 73)",
 "PerformanceEvent('Note OFF', 37)",
 "PerformanceEvent('Note OFF', 76)",
 "PerformanceEvent('Note OFF', 33)",
 "PerformanceEvent('Note OFF', 74)",
 "PerformanceEvent('Note OFF', 77)",
 "PerformanceEvent('Note OFF', 74)",
 "PerformanceEvent('Note OFF', 74)",
 "PerformanceEvent('Note OFF', 75)",
 "PerformanceEvent('Note OFF', 76)",
 "PerformanceEvent('Note OFF', 76)",
 "PerformanceEvent('Note OFF', 79)",
 "PerformanceEvent('Note OFF', 67)",
 "PerformanceEvent('Note OFF', 76)",
 "PerformanceEvent('Note OFF', 76)",
 "PerformanceEvent('Note OFF', 74)",
 "PerformanceEvent('Note OFF', 76)",
 "PerformanceEvent('Note OFF', 76)",
 "PerformanceEvent('Note OFF', 79)",
 "PerformanceEvent('Note OFF', 79)",
 "PerformanceEvent('Note OFF', 78)",
 "PerformanceEvent('Note OFF', 78)",
 

In [19]:
for timestep in tqdm(list(range(INPUT_SEQ_LEN, len(generated)))):
    start_index = timestep - (INPUT_SEQ_LEN)
    sequence_for_prediction = generated[start_index:timestep]
    next_step = sample(model, sequence_for_prediction, 1.5)
    generated[timestep] = next_step

decoded_generated_sequence = np.argmax(generated[INPUT_SEQ_LEN:],axis=1)
index_of_nonzeros = np.where(decoded_generated_sequence != 0)
decoded_generated_sequence_nozeros = decoded_generated_sequence[index_of_nonzeros]

new_path = os.path.join(samples_dir, "temp_%s_%s.mid" %(temperature, i))
path = my_encoder.encoder.decode(decoded_generated_sequence_nozeros, path=new_path)

roll = pypianoroll.Multitrack(new_path).tracks[0]

plt.figure(figsize=(14,8))
ax = plt.gca()
pypianoroll.plot_pianoroll(ax, roll.pianoroll)
plt.show()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 295/295 [00:05<00:00, 57.72it/s]


ValueError: Cannot get beat timings to quantize pianoroll.

In [20]:
my_encoder.printevents(np.argmax(generated,axis=1))

["PerformanceEvent('Note ON', 66)",
 "PerformanceEvent('Time Shift', 25)",
 "PerformanceEvent('Note OFF', 66)",
 "PerformanceEvent('Note ON', 66)",
 "PerformanceEvent('Time Shift', 26)",
 "PerformanceEvent('Note OFF', 66)",
 "PerformanceEvent('Note ON', 66)",
 "PerformanceEvent('Time Shift', 25)",
 "PerformanceEvent('Note OFF', 66)",
 "PerformanceEvent('Note ON', 64)",
 "PerformanceEvent('Time Shift', 26)",
 "PerformanceEvent('Note OFF', 64)",
 "PerformanceEvent('Time Shift', 25)",
 "PerformanceEvent('Note ON', 69)",
 "PerformanceEvent('Time Shift', 26)",
 "PerformanceEvent('Note OFF', 69)",
 "PerformanceEvent('Note ON', 74)",
 "PerformanceEvent('Time Shift', 49)",
 "PerformanceEvent('Note OFF', 74)",
 "PerformanceEvent('Note ON', 69)",
 "PerformanceEvent('Time Shift', 25)",
 "PerformanceEvent('Note OFF', 69)",
 "PerformanceEvent('Time Shift', 25)",
 "PerformanceEvent('Note ON', 69)",
 "PerformanceEvent('Time Shift', 26)",
 "PerformanceEvent('Note OFF', 69)",
 "PerformanceEvent('Note O

# Inspect .dat file

In [60]:
dataset = "HOOKTHEORY_DATASET_MAGENTA_40k"
dataset = datasets[dataset]
dpath = dataset['path']
dataset = np.memmap(dpath, mode="r",
                    dtype="uint8", shape=dataset['shape'])

In [61]:
last = np.unique(np.argmax(dataset[:,-1],axis=1),return_counts=True)

In [63]:
last

(array([  0, 100, 102, 104, 105, 107, 109, 110, 111, 112, 113, 114, 115,
        116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128,
        129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141,
        142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154,
        155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
        169, 170, 172, 174], dtype=int64),
 array([ 182,    1,    1,    2,    7,    4,   26,   14,    3,   42,    7,
          85,    5,   47,  167,    4,  171,   24,  461,  141,   31,  974,
          79, 1472,   83, 1051, 3462,   92, 2958,  127, 4088, 1136,  265,
        3459,  235, 3165,  179, 1900, 4393,  123, 3027,  127, 3435,  942,
         155, 1744,  113,  897,   67,  461,  649,   31,  316,   27,  314,
          63,   13,  102,    7,   53,    2,   24,   30,    1,   13,   11,
           4,    3,    1], dtype=int64))

In [62]:
np.save("hooktheory_40k_last.npy", last)

## What about folk dataset?

In [64]:
dataset = "FOLK_DATASET_MAGENTA_200k"
dataset = datasets[dataset]
dpath = dataset['path']
dataset = np.memmap(dpath, mode="r",
                    dtype="uint8", shape=dataset['shape'])

In [65]:
last = np.unique(np.argmax(dataset[:50000,-1],axis=1),return_counts=True)

In [67]:
last

(array([  0,   4,   8,  21,  24,  26,  28,  29,  31,  33,  34,  35,  36,
         37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,
         50,  51,  52,  53,  54,  55,  57,  58,  59,  60,  61,  62,  63,
         64,  65,  67,  69,  70,  72,  74, 112, 114, 116, 117, 119, 121,
        122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134,
        135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147,
        148, 149, 150, 151, 152, 153, 155, 157, 158, 160, 162, 163, 165,
        181, 182, 185, 186, 189, 190, 193, 194, 198, 202, 206, 207, 210,
        211, 214, 215, 223, 227, 248, 252], dtype=int64),
 array([28653,     1,     1,    44,     1,     1,     2,     6,     6,
           25,     7,     1,   123,     1,   164,     1,   142,   453,
            5,   422,     2,   658,   308,    35,   820,     6,   596,
           26,   364,   841,     3,   572,   608,   238,    12,   303,
            3,   145,    14,    35,    70,    26,    16,     4,     4,
   

In [66]:
np.save("folk_40k_last.npy", last)